In [1]:
import sys

sys.path.append('../')

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from probabilistic_covshift.constants.automl_constants import AutoMLConfig as AutoMLConfig
from probabilistic_covshift.constants.automl_constants import H2OServerInfo as H2OServerInfo
from probabilistic_covshift.constants.main_constants import OriginFeatures as OriginFeatures
from probabilistic_covshift.constants.main_constants import WeightFeatures as WeightFeatures
from probabilistic_covshift.probabilistic_classification_covshift import ProbabilisticClassification

In [2]:
spark = SparkSession.builder.appName('main').master('local[4]').getOrCreate()

In [7]:
source_df = spark.createDataFrame([
    ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0),
    ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), 
    ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0),
    ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0),
    ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0),
    ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0),
    ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0),
    ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0),
    ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0),
    ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), 
    ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0),
    ('A', 88.9, 50.0, 15, 'E', 20.0), ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0),
    ('B', 48.9, 40.0, 55, 'P', 20.0), ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0),
    ('B', 98.9, 50.0, 15, 'E', 20.0), ('A', 38.9, 40.0, 55, 'E', 10.0), ('A', 88.9, 50.0, 15, 'E', 20.0),
    ('B', 18.9, 40.0, 55, 'P', 30.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 48.9, 40.0, 55, 'P', 20.0),
    ('B', 58.9, 50.0, 15, 'E', 30.0), ('A', 38.9, 40.0, 55, 'P', 10.0), ('B', 98.9, 50.0, 15, 'E', 20.0)], 
    ['col_a', 'col_b', 'col_c', 'col_d', 'col_e', 'col_f'])

In [8]:
target_df = spark.createDataFrame([
    ('A', 48.9, 40.0, 55, 'E', 10.0), ('A', 58.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 30.0),
    ('A', 48.9, 50.0, 15, 'E', 10.0), ('B', 58.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 30.0),
    ('A', 48.9, 40.0, 55, 'P', 10.0), ('B', 58.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 50.0, 15, 'E', 20.0),
    ('A', 48.9, 40.0, 55, 'E', 10.0), ('A', 58.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 30.0),
    ('A', 48.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 30.0),
    ('A', 48.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 50.0, 15, 'E', 20.0),
    ('A', 58.9, 40.0, 55, 'E', 10.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 40.0, 55, 'P', 30.0),
    ('A', 58.9, 50.0, 15, 'E', 10.0), ('B', 148.9, 140.0, 55, 'P', 10.0), ('B', 38.9, 50.0, 15, 'E', 30.0),
    ('A', 58.9, 40.0, 55, 'P', 110.0), ('B', 198.9, 150.0, 15, 'E', 10.0), ('B', 38.9, 50.0, 15, 'E', 20.0),
    ('A', 58.9, 40.0, 55, 'E', 110.0), ('A', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 140.0, 55, 'P', 30.0),
    ('A', 58.9, 50.0, 15, 'E', 110.0), ('B', 38.9, 40.0, 55, 'P', 10.0), ('B', 38.9, 150.0, 15, 'E', 30.0),
    ('A', 58.9, 40.0, 55, 'P', 110.0), ('B', 38.9, 50.0, 15, 'E', 10.0), ('B', 38.9, 150.0, 15, 'E', 20.0)], 
    ['col_a', 'col_b', 'col_c', 'col_d', 'col_e', 'col_f'])

In [9]:
automl_conf = {
    AutoMLConfig.DATA: {
        AutoMLConfig.LABEL_COL: OriginFeatures.ORIGIN,
        AutoMLConfig.WEIGHT_COL: WeightFeatures.WEIGHT,
        AutoMLConfig.BASE_TABLE_PATH: 'data/base_table.parquet',
    },
    AutoMLConfig.SERVER_CONN_INFO: {
        H2OServerInfo.IP: 'localhost',
        H2OServerInfo.PORT: '54321'
    },
    AutoMLConfig.CROSS_VAL: {
        AutoMLConfig.FOLD_COL: "fold",
        AutoMLConfig.NFOLDS: 8,
    },
    AutoMLConfig.MODELING: {
        AutoMLConfig.MAX_RUNTIME_SECS: 3600,
        AutoMLConfig.MAX_MODELS: 10,
        AutoMLConfig.STOPPING_METRIC: 'logloss',
        AutoMLConfig.SORT_METRIC: 'logloss'
    },
    AutoMLConfig.EXCLUDE_ALGOS: [
        "StackedEnsemble",
        "DeepLearning"
    ],
    AutoMLConfig.MODEL: {
        AutoMLConfig.BEST_MODEL_PATH: 'data/model/'
    },
    AutoMLConfig.SEED: 23
}

In [10]:
pc = ProbabilisticClassification(source_df, target_df, automl_conf)
pc.run()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,6 mins 27 secs
H2O cluster timezone:,Asia/Jakarta
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,2 months and 4 days
H2O cluster name:,H2O_from_python_albertus_kelvin_xeikza
H2O cluster total nodes:,1
H2O cluster free memory:,3.525 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Base table inferred column types: {'col_b': 'real', 'col_c': 'int', 'col_d': 'int', 'col_f': 'int', 'origin': 'enum'}


INFO:probabilistic_covshift.automl.trainer:Base table inferred column types: {'col_b': 'real', 'col_c': 'int', 'col_d': 'int', 'col_f': 'int', 'origin': 'enum'}


AutoML progress: |
18:13:13.150: Fold column fold will be used for cross-validation. nfolds parameter will be ignored.

████████████████
18:13:22.223: Skipping training of model GBM_5_AutoML_20200325_181313 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200325_181313.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 84.0.


████████████████████████████████████████| 100%


...

Loaded model: 


INFO:probabilistic_covshift.automl.predictor:Loaded model: 


gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,source,target,id,col_b,col_c,col_d,col_f,BiasTerm
source,0.847295,0.152705,0,-0.331068,-0.650236,-0.0827325,-0.135599,-0.513907
source,0.998822,0.00117789,1,-4.07352,-0.134245,-0.141808,-1.87937,-0.513907
source,0.998708,0.00129168,2,-4.86913,-0.664431,-0.38018,-0.222875,-0.513907
source,0.615075,0.384925,3,0.238943,0.0949844,0.106115,-0.394833,-0.513907
source,0.994023,0.00597698,4,1.22753,-1.38553,-0.782682,-3.65926,-0.513907
source,0.990932,0.00906833,5,-0.31839,-0.755529,-0.611121,-2.49491,-0.513907
source,0.847295,0.152705,6,-0.331068,-0.650236,-0.0827325,-0.135599,-0.513907
source,0.998822,0.00117789,7,-4.07327,-0.134492,-0.141814,-1.87937,-0.513907
source,0.847295,0.152705,8,-0.331068,-0.650236,-0.0827325,-0.135599,-0.513907
source,0.998822,0.00117789,9,-4.07352,-0.134245,-0.141808,-1.87937,-0.513907



<class 'h2o.frame.H2OFrame'>


In [19]:
import h2o

In [20]:
python_obj = [1, 2, 2.5, -100.9, 0]
frame = h2o.H2OFrame(python_obj)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [21]:
frame.shape[0]

5

In [24]:
python_obj = [0, 0, 2.5, -100.9, 0]
frameb = h2o.H2OFrame(python_obj)
frame=frame.cbind(frameb.set_names(['tes']))

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [25]:
frame['tes']

tes
0
0
2.5
-100.9
0


In [26]:
h2o.remove_all()